In [ ]:
## recreating allele freq

In [ ]:
## shoul i cehck if i can get certain metrics from trees?

In [1]:
# once i have wholegenome_offset.trees
import pandas as pd
import tskit
import allel
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tsinfer
import pyslim
import os
from collections import defaultdict

In [2]:
## calculate allele freq 

In [ ]:
output_vcf_fixpos = snakemake.input['output_vcf_fixpos'] 
pos_vcf_og = snakemake.input['pos_vcf_og'] 
output_allele_freq = snakemake.output['allele_freq'] 



In [3]:
file_names = [
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima0/subp0_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima0/subp1_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima0/subp2_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima0/subp3_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima0/subp4_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima1/subp0_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima1/subp1_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima1/subp2_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima1/subp3_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima1/subp4_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima2/subp0_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima2/subp1_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima2/subp2_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima2/subp3_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima2/subp4_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima3/subp0_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima3/subp1_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima3/subp2_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima3/subp3_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima3/subp4_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima4/subp0_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima4/subp1_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima4/subp2_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima4/subp3_vcf_output_rp.vcf",
    "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima4/subp4_vcf_output_rp.vcf"
]

In [5]:
path = '/home/tbellagio/scratch/slim_grenenet/'

In [20]:
file = "results/arq_mediumfreq_onehpoly_highbeta/strongsel/optima0/subp0_vcf_output_rp.vcf"

In [21]:
path_complete = path+file

In [22]:
if os.path.exists(path_complete) and os.path.getsize(path_complete) <= 1:
    print('empty_vcf')

elif os.path.exists(path_complete) and os.path.getsize(path_complete) > 1:
    print('stg')

empty_vcf


In [13]:
vcf = allel.read_vcf('../treeseq/greneNet_final_v1.1.recode.vcf.gz', fields= 'variants/POS')
pos_vcf_og = pd.DataFrame(data = {'pos':vcf['variants/POS'] })
pos_vcf_og.to_csv(path + 'data/pos_vcf_og.csv', index=None)

In [13]:
vcf = allel.read_vcf('../treeseq/greneNet_final_v1.1.recode.vcf.gz', fields= 'variants/POS')
pos_vcf_og = pd.DataFrame(data = {'pos':vcf['variants/POS'] })
pos_vcf_og.to_csv(path + 'data/pos_vcf_og.csv', index=None)

In [15]:
pos_vcf_og = pd.read_csv(pos_vcf_og)

In [16]:
pos_vcf_og[name] = np.nan

NameError: name 'name' is not defined

In [76]:
def extract_allele_freq(samples, geno_array, pos, name):
    total_alleles = len(samples) * 2 
    alt_freq = geno_array.sum(axis=2).sum(axis=1) / total_alleles
    alt_allele_freq = pd.DataFrame(data = {'pos': pos, name: alt_freq})
    return alt_allele_freq

In [88]:
for i in output_vcf_fixpos:
    print(i)
    name = i.split('/')[-2] + '_' + i.split('/')[-1][0:5]
    if os.path.exists(i) and os.path.getsize(i) <= 1:
        print('empty_vcf')
        pos_vcf_og[name] = np.nan
    elif os.path.exists(i) and os.path.getsize(i) > 1:
        vcf = allel.read_vcf(i)
        samples = vcf['samples']
        geno_array = vcf['calldata/GT']
        pos = vcf['variants/POS']
        alt_allele_freq = extract_allele_freq(samples,geno_array, pos, name)
        pos_vcf_og = pos_vcf_og.merge(alt_allele_freq, on ='pos', how='outer')
pos_vcf_og.to_csv(output_allele_freq)

,pos,result_selection_var1.3031_optima2.5131.vcfoutput_selection_x,result_selection_var1.3031_optima2.5131.vcfoutput_selection_y
0,291,0.038265,NaN
1,346,0.214286,0.145570
2,349,0.007653,NaN
3,352,NaN,0.025316
4,353,0.056122,0.107595
...,...,...,...
3235475,119146054,NaN,0.006329
3235476,119146061,NaN,NaN
3235477,119146113,NaN,NaN
3235478,119146118,0.165816,0.322785


In [ ]:
import glob
import pandas as pd
import os
import tskit



path = '/home/tbellagio/scratch/slim_grenenet/'


# Use the glob module to search for files in subfolders
pattern = os.path.join(path, '**', '*_tree_output.trees')
trees = glob.glob(pattern, recursive=True)



pop_size = dict()
for i in trees:
    print(i)
    if os.path.getsize(i) > 1:
        tree = tskit.load(i)
        pop_s = len(tree.tables.individuals)
        pop_size[i] = pop_s
    else:
        pop_size[i] = 0

pd.DataFrame([pop_size]).T.to_csv('pop_sizes.csv')

In [4]:
pop_size = pd.read_csv('pop_sizes.csv')

In [7]:
pop_size[pop_size['0']==0]

,Unnamed: 0,0
10,/home/tbellagio/scratch/slim_grenenet/results/...,0
11,/home/tbellagio/scratch/slim_grenenet/results/...,0
12,/home/tbellagio/scratch/slim_grenenet/results/...,0
13,/home/tbellagio/scratch/slim_grenenet/results/...,0
14,/home/tbellagio/scratch/slim_grenenet/results/...,0
...,...,...
1785,/home/tbellagio/scratch/slim_grenenet/results/...,0
1786,/home/tbellagio/scratch/slim_grenenet/results/...,0
1787,/home/tbellagio/scratch/slim_grenenet/results/...,0
1788,/home/tbellagio/scratch/slim_grenenet/results/...,0


In [26]:
# Use the glob module to search for files in subfolders
pattern = os.path.join(path, '**', 'allele_freq.csv')
allele_freq_files = glob.glob(pattern, recursive=True)

In [27]:
len(allele_freq_files)

48

In [23]:
for i in allele_freq_files:
    if os.path.getsize(i) != 133292499:
        print(i)
        print(os.path.getsize(i))

/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_twentypoly_lowbeta/lowsel/allele_freq.csv
807284550
/home/tbellagio/scratch/slim_grenenet/results/arq_lowfreq_monogen_highbeta/strongsel/allele_freq.csv
1770104976
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_monogen_lowbeta/strongsel/allele_freq.csv
1567941385
/home/tbellagio/scratch/slim_grenenet/results/arq_lowfreq_onehpoly_highbeta/lowsel/allele_freq.csv
3337535574
/home/tbellagio/scratch/slim_grenenet/results/arq_lowfreq_twentypoly_highbeta/lowsel/allele_freq.csv
1698297950
/home/tbellagio/scratch/slim_grenenet/results/arq_lowfreq_twentypoly_highbeta/moderatesel/allele_freq.csv
1664068319
/home/tbellagio/scratch/slim_grenenet/results/arq_lowfreq_fivepoly_lowbeta/moderatesel/allele_freq.csv
1768562305
/home/tbellagio/scratch/slim_grenenet/results/arq_lowfreq_fivepoly_lowbeta/strongsel/allele_freq.csv
1757685525
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/lowsel/allele_fre

In [ ]:
305680693
807284550

In [46]:
allele_freq = pd.read_csv('/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/allele_freq.csv')

In [47]:
allele_freq.pos.value_counts()

pos
12314751    125
8373311      80
9362109      64
10487668     64
17136607     64
           ... 
7811645       1
7811657       1
7811662       1
7811718       1
26975288      1
Name: count, Length: 3085684, dtype: int64

In [33]:
allele_freq[allele_freq['pos'] == 8373311]

,Unnamed: 0,pos,optima0_subp0,optima0_subp1,optima0_subp2,optima0_subp3,optima0_subp4,optima1_subp0,optima1_subp1,optima1_subp2,...,optima3_subp0,optima3_subp1,optima3_subp2,optima3_subp3,optima3_subp4,optima4_subp0,optima4_subp1,optima4_subp2,optima4_subp3,optima4_subp4
811635,811635,8373311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811636,811636,8373311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811637,811637,8373311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811638,811638,8373311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811639,811639,8373311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814130,814130,8373311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
814131,814131,8373311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
814132,814132,8373311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
814133,814133,8373311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
allele_freq

,Unnamed: 0,pos,optima0_subp0,optima0_subp1,optima0_subp2,optima0_subp3,optima0_subp4,optima1_subp0,optima1_subp1,optima1_subp2,...,optima3_subp0,optima3_subp1,optima3_subp2,optima3_subp3,optima3_subp4,optima4_subp0,optima4_subp1,optima4_subp2,optima4_subp3,optima4_subp4
0,0,291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.022778,0.052778,NaN,NaN,NaN,NaN,NaN
1,1,346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.325000,0.246111,NaN,NaN,NaN,NaN,NaN
2,2,349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.007778,NaN,NaN,NaN,NaN,NaN,NaN
3,3,352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.032222,0.036111,NaN,NaN,NaN,NaN,NaN
4,4,353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.072778,0.080556,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049241,4049241,26975208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.036667,0.025556,NaN,NaN,NaN,NaN,NaN
4049242,4049242,26975215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4049243,4049243,26975267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.012778,0.005556,NaN,NaN,NaN,NaN,NaN
4049244,4049244,26975272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.398889,0.459444,NaN,NaN,NaN,NaN,NaN


In [12]:
# Use the glob module to search for files in subfolders
pattern = os.path.join(path, '**', '*_vcf_output_rp.vcf')
vcfs = glob.glob(pattern, recursive=True)

In [15]:
sizes_vcfs = []
for i in vcfs:
    sizes_vcfs.append(os.path.getsize(i))

In [17]:
pd.Series(sizes_vcfs).value_counts()

0              1393
11056845485       1
11389121373       1
11197713565       1
11155652884       1
               ... 
10796901595       1
10792573363       1
10984333266       1
10832239582       1
11223506507       1
Name: count, Length: 408, dtype: int64

In [35]:
os.listdir('/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/')

['optima4', 'optima3', 'optima0', 'optima2', 'allele_freq.csv', 'optima1']

In [38]:
# Use the glob module to search for files in subfolders
pattern = os.path.join('/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/', '**', '*_vcf_output_rp.vcf')
vcfs_one = glob.glob(pattern, recursive=True)

In [40]:
vcfs_one[:10]

['/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima4/subp4_vcf_output_rp.vcf',
 '/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima4/subp2_vcf_output_rp.vcf',
 '/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima4/subp3_vcf_output_rp.vcf',
 '/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima4/subp0_vcf_output_rp.vcf']

In [50]:
import pandas as pd
import allel
import os
import numpy as np

output_vcf_fixpos = vcfs_one  #[:4]

In [60]:
pos_vcf_og_og = pd.read_csv(path + 'data/pos_vcf_og.csv')

In [206]:
def extract_allele_freq(samples, geno_array, pos, chrom, name):
    total_alleles = len(samples) * 2 
    alt_freq = geno_array.sum(axis=2).sum(axis=1) / total_alleles
    chrom_pos = pd.Series(chrom.astype(str)) + '_' +  pd.Series(pos.astype(str))
    alt_allele_freq = pd.DataFrame(data = {'chrom_pos': chrom_pos, name: alt_freq})
    return alt_allele_freq

In [153]:
vcf = allel.read_vcf('/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima3/subp4_vcf_output_rp.vcf')
samples = vcf['samples']
geno_array = vcf['calldata/GT']
pos = vcf['variants/POS']
chrom = vcf['variants/CHROM']
alt_allele_freq = extract_allele_freq(samples,geno_array, pos, chrom, name)


In [157]:
alt_allele_freq['chrom_pos'] = alt_allele_freq['chrom'].astype(str) + '_' + alt_allele_freq['pos'].astype(str)

In [169]:
alt_allele_freq = alt_allele_freq.drop(['pos', 'chrom'],axis=1)

In [159]:
alt_allele_freq['chrom_pos'].value_counts()

chrom_pos
1_291         1
4_6091858     1
4_6091726     1
4_6091730     1
4_6091752     1
             ..
2_7937958     1
2_7938013     1
2_7938021     1
2_7938040     1
5_26975288    1
Name: count, Length: 3031595, dtype: int64

In [204]:
pos_vcf_og = pd.read_csv(path + 'data/pos_vcf_og_nooffset.csv')

In [196]:
len(pos_vcf_og)

3235480

In [197]:
len(alt_allele_freq)

3031595

In [198]:
pos_vcf_og = pos_vcf_og.merge(alt_allele_freq, on ='chrom_pos', how='outer')

In [200]:
len(pos_vcf_og)

3235480

In [199]:
pos_vcf_og['chrom_pos'].value_counts()

chrom_pos
1_291         1
4_6060556     1
4_6060569     1
4_6060576     1
4_6060580     1
             ..
2_7962987     1
2_7962989     1
2_7962993     1
2_7962994     1
5_26975288    1
Name: count, Length: 3235480, dtype: int64

In [202]:
~alt_allele_freq['chrom_pos'].isin(pos_vcf_og['chrom_pos'])

0          False
1          False
2          False
3          False
4          False
           ...  
3031590    False
3031591    False
3031592    False
3031593    False
3031594    False
Name: chrom_pos, Length: 3031595, dtype: bool

In [203]:
alt_allele_freq[~alt_allele_freq['chrom_pos'].isin(pos_vcf_og['chrom_pos'])]

,optima1_subp0,chrom_pos


In [ ]:
def extract_allele_freq(samples, geno_array, pos, chrom, name):
    total_alleles = len(samples) * 2 
    alt_freq = geno_array.sum(axis=2).sum(axis=1) / total_alleles
    chrom_pos = pd.Series(chrom.astype(str)) + '_' +  pd.Series(pos.astype(str))
    alt_allele_freq = pd.DataFrame(data = {'chrom_pos': chrom_pos, name: alt_freq})
    return alt_allele_freq

In [53]:
for i in output_vcf_fixpos:
    print(i)
    name = i.split('/')[-2] + '_' + i.split('/')[-1][0:5]
    if os.path.exists(i) and os.path.getsize(i) <= 1:
        print('empty_vcf')
        pos_vcf_og[name] = np.nan
    elif os.path.exists(i) and os.path.getsize(i) > 1:
        vcf = allel.read_vcf(i)
        samples = vcf['samples']
        geno_array = vcf['calldata/GT']
        pos = vcf['variants/POS']
        chrom = vcf['variants/CHROM']
        alt_allele_freq = extract_allele_freq(samples,geno_array, pos, chrom, name)
        pos_vcf_og = pos_vcf_og.merge(alt_allele_freq, on ='pos', how='outer')
pos_vcf_og.to_csv(output_allele_freq)

print(output_allele_freq)
pos_vcf_og.to_csv(output_allele_freq)

/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima4/subp4_vcf_output_rp.vcf
empty_vcf
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima4/subp2_vcf_output_rp.vcf
empty_vcf
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima4/subp3_vcf_output_rp.vcf
empty_vcf
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima4/subp0_vcf_output_rp.vcf
empty_vcf
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima4/subp1_vcf_output_rp.vcf
empty_vcf
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima3/subp1_vcf_output_rp.vcf
empty_vcf
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima3/subp0_vcf_output_rp.vcf
empty_vcf
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly

NameError: name 'output_allele_freq' is not defined

In [148]:
vcf['variants/CHROM']

array(['1', '1', '1', ..., '5', '5', '5'], dtype=object)

In [149]:
alt_allele_freq

,pos,optima3_subp4
0,291,0.052778
1,346,0.246111
2,352,0.036111
3,353,0.080556
4,363,0.061111
...,...,...
3031590,26975206,0.016667
3031591,26975208,0.025556
3031592,26975267,0.005556
3031593,26975272,0.459444


In [ ]:
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima3/subp4_vcf_output_rp.vcf


In [73]:
vcf_og = allel.read_vcf('/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima3/subp4_vcf_output.vcf')

In [74]:
pos_og = vcf_og['variants/POS']

In [99]:
fulldfog = pd.DataFrame(data = {'pos_test': pos_og, 'chr': vcf_og['variants/CHROM']})

In [113]:
fulldfog[fulldfog['pos_test']==42742422]

,pos_test,chr
1128878,42742422,1


In [75]:
pd.Series(pos_og).value_counts()

291          1
79677648     1
79677516     1
79677520     1
79677542     1
            ..
38365629     1
38365684     1
38365692     1
38365711     1
119146134    1
Name: count, Length: 3031595, dtype: int64

In [86]:
vcf_test = allel.read_vcf('test_right_pos.vcf')

In [92]:
vcf_test['variants/CHROM']

array(['1', '1', '1', ..., '5', '5', '5'], dtype=object)

In [87]:
pos_test = vcf_test['variants/POS']

In [109]:
comparision = pd.DataFrame(data = {'pos_og': pos_og, 'pos_test': pos_test})

In [139]:
comparision[comparision['pos_test'] == 12314751]['pos_og'].values

array([ 12314751,  42742422,  62440711,  85900541, 104485597], dtype=int32)

In [105]:
len(pos_og)

3031595

In [107]:
len(pos_test)

3031595

In [94]:
fulldf = pd.DataFrame(data = {'pos_test': pos_test, 'chr': vcf_test['variants/CHROM']})

In [96]:
fulldf[fulldf['pos_test']==12314751]

,pos_test,chr
285164,12314751,1
1128878,12314751,2
1585467,12314751,3
2227515,12314751,4
2641004,12314751,5


In [88]:
pd.Series(pos_test).value_counts()

12314751    5
8513910     4
8373311     4
16789958    4
10796768    4
           ..
7771132     1
7771156     1
7771176     1
7771231     1
26975288    1
Name: count, Length: 2900008, dtype: int64

In [84]:
pd.Series(pos_test).value_counts()[pd.Series(pos_test).value_counts() != 1].sum()

260062

In [85]:
260062 - 128475

131587

In [80]:
3031595 - 2900008

131587

In [72]:
pd.Series(pos).value_counts()

12314751    5
8513910     4
8373311     4
16789958    4
10796768    4
           ..
7771132     1
7771156     1
7771176     1
7771231     1
26975288    1
Name: count, Length: 2900008, dtype: int64

In [ ]:
/home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima3/subp4_vcf_output.vcf > test_right_pos.vcf

In [ ]:
awk -F'\t' 'BEGIN{OFS="\t"} $1 ~ /^#/ {print} $1 !~ /^#/ {pos=$2; if (pos > 30427671 && pos <= 50125959) {pos -= 30427671; $1="2"} else if (pos > 50125959 && pos <= 73585789) {pos -= 50125960; $1="3"} else if (pos > 73585789 && pos <= 92170845) {pos -= 73585790; $1="4"} else if (pos > 92170845 && pos <= 119146348) {pos -= 92170846; $1="5"} $2=pos; print}' 

In [ ]:
12314751

In [ ]:
awk 'BEGIN {OFS="\t"} /^#/ {print} !/^#/   
                {if ($2 > 30427671 && $2 <= 50125959) {$2 = $2 - 30427671; $1="2"}; 
                else if($2 > 50125959 && $2 <= 73585789) {$2 = $2 - 50125960; $1="3"};    
                else if($2 > 73585789 && $2 <= 92170845) {$2 = $2 - 73585790; $1="4"}; 
                else if($2 > 92170845 && $2 <= 119146348) {$2 = $2 - 92170846; $1="5"};        
                print}' /home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima3/subp4_vcf_output.vcf > test_right_pos.vcf

In [ ]:
awk 'BEGIN {OFS="\t"} /^#/ {print} !/^#/ {if ($2 > 30427671 && $2 <= 50125959) {$2 = $2 - 30427671; $1="2"}; else if($2 > 50125959 && $2 <= 73585789) {$2 = $2 - 50125960; $1="3"}; else if($2 > 73585789 && $2 <= 92170845) {$2 = $2 - 73585790; $1="4"}; else if($2 > 92170845 && $2 <= 119146348) {$2 = $2 - 92170846; $1="5"}; print}' /home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima3/subp4_vcf_output.vcf > test_right_pos.vcf

In [ ]:
awk 'BEGIN {OFS="\t"} /^#/ {print} !/^#/   
                {if ($1 == "2") $2 = $2 + 30427671; 
                else if ($1 == "3") $2 = $2 + 50125960;    
                else if ($1 == "4") $2 = $2 + 73585790; 
                else if ($1 == "5") $2 = $2 + 92170846;        
                print}' greneNet_final_v1.1_nohet.recode.vcf > wholegenome_offset_nohet.vcf

In [ ]:
##contig=<ID=Chr1,length=30427671>
##contig=<ID=Chr2,length=19698289>
##contig=<ID=Chr3,length=23459830>
##contig=<ID=Chr4,length=18585056>
##contig=<ID=Chr5,length=26975502>

In [90]:
12314751

12314751

In [89]:
30427671 + 19698289

50125960

In [ ]:
greneNet_final_v1.1.recode.vcf: og vcf file

wholegenome_offset.vcf : same vcf file but with the positions values changed with the awk command

In [ ]:
wholegenome_offset.vcf

In [141]:
vcf_og_offset = allel.read_vcf(path + 'data/wholegenome_offset.vcf',fields=['variants/POS', 'variants/CHROM'])

In [142]:
pos_og_offset = vcf_og_nooffset['variants/POS']

In [143]:
len(pos_og_offset)

3235480

In [144]:
offset_trans = pd.DataFrame({'offset': pos_og_offset, 'nooffset':pos_og_og })

In [187]:
offset_trans[offset_trans['offset'].isin([30484595])]

,offset,nooffset
827765,30484595,56924


In [119]:
vcf_og_og = allel.read_vcf('../treeseq/greneNet_final_v1.1.recode.vcf.gz',fields=['variants/POS', 'variants/CHROM'])

In [121]:
pos_og_og = vcf_og_og['variants/POS']

In [188]:
pos_og_chr = vcf_og_og['variants/CHROM']

In [189]:
pos_og_chr

array(['1', '1', '1', ..., '5', '5', '5'], dtype=object)

In [190]:
pos_og_full = pd.DataFrame({'pos': pos_og_og, 'chr':pos_og_chr })

In [193]:
pos_og_full[pos_og_full['pos'] == 56924]

,pos,chr
827765,56924,2


In [78]:
pd.Series(pos_test).value_counts()

12314751    5
8513910     4
8373311     4
16789958    4
10796768    4
           ..
7771132     1
7771156     1
7771176     1
7771231     1
26975288    1
Name: count, Length: 2900008, dtype: int64

In [122]:
len(pos_og_og)

3235480

In [123]:
len(pos_og)

3031595

In [114]:
comparision[comparision['pos_test'] == 12314751]

,pos_og,pos_test
285164,12314751,12314751
1128878,42742422,12314751
1585467,62440711,12314751
2227515,85900541,12314751
2641004,104485597,12314751


In [115]:
42742422 > 30427671

True

In [186]:
56924 + 30427671

30484595

In [69]:
awk 'BEGIN {
    OFS="\t"
}

/^#/ {
    print
}

!/^#/ {
    if ($2 > 30427671 && $2 <= 50125959) {
        $2 = $2 - 30427671
        $1 = "2"
    } else if ($2 > 50125959 && $2 <= 73585789) {
        $2 = $2 - 50125960
        $1 = "3"
    } else if ($2 > 73585789 && $2 <= 92170845) {
        $2 = $2 - 73585790
        $1 = "4"
    } else if ($2 > 92170845 && $2 <= 119146348) {
        $2 = $2 - 92170846
        $1 = "5"
    }
    print
}' /home/tbellagio/scratch/slim_grenenet/results/arq_mediumfreq_fivepoly_highbeta/moderatesel/optima3/subp4_vcf_output.vcf > test_right_pos.vcf

,pos,optima3_subp4
0,291,0.052778
1,346,0.246111
2,352,0.036111
3,353,0.080556
4,363,0.061111
...,...,...
3031590,26975206,0.016667
3031591,26975208,0.025556
3031592,26975267,0.005556
3031593,26975272,0.459444


In [68]:
alt_allele_freq['pos'].value_counts()

pos
12314751    5
8513910     4
8373311     4
16789958    4
10796768    4
           ..
7771132     1
7771156     1
7771176     1
7771231     1
26975288    1
Name: count, Length: 2900008, dtype: int64

In [65]:
merge = pos_vcf_og_og.merge(alt_allele_freq, on ='pos',how='outer')

In [67]:
merge['pos'].value_counts()

pos
12314751    25
8373311     20
8383382     16
19404955    16
16134796    16
            ..
7811645      1
7811657      1
7811662      1
7811718      1
26975288     1
Name: count, Length: 3085684, dtype: int64

In [59]:
pos_vcf_og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4049246 entries, 0 to 4049245
Data columns (total 26 columns):
 #   Column         Dtype  
---  ------         -----  
 0   pos            int64  
 1   optima4_subp4  float64
 2   optima4_subp2  float64
 3   optima4_subp3  float64
 4   optima4_subp0  float64
 5   optima4_subp1  float64
 6   optima3_subp1  float64
 7   optima3_subp0  float64
 8   optima3_subp3  float64
 9   optima3_subp4  float64
 10  optima3_subp2  float64
 11  optima0_subp0  float64
 12  optima0_subp1  float64
 13  optima0_subp4  float64
 14  optima0_subp2  float64
 15  optima0_subp3  float64
 16  optima2_subp3  float64
 17  optima2_subp2  float64
 18  optima2_subp4  float64
 19  optima2_subp1  float64
 20  optima2_subp0  float64
 21  optima1_subp3  float64
 22  optima1_subp2  float64
 23  optima1_subp4  float64
 24  optima1_subp1  float64
 25  optima1_subp0  float64
dtypes: float64(25), int64(1)
memory usage: 803.2 MB
